## SQL Alchemy Class definition

In [3]:
from sqlalchemy import Column, Integer, String, TEXT, ARRAY
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import create_engine

Base = declarative_base()

engine = create_engine('sqlite:///../articles.db')
Base.metadata.create_all(engine)

class Article(Base):
    __tablename__ = 'locations'

    id = Column(Integer, primary_key=True)
    
    title = Column(String(80), nullable=False)
    url = Column(String(80), nullable=False)
    text = Column(TEXT, nullable=False)
    most_common = Column(ARRAY(String), nullable=False)
    
    def __init__(self, title, text, url, most_common):
        self.title = title
        self.text = text
        self.url = url
        self.most_common = most_common
        

## Scraper code

In [ ]:
def get_text(soup):
    text = ""
    for paragraph in soup.find_all('p'):
        if not paragraph.has_attr('class') and not paragraph.findChildren('b'):
            text += paragraph.get_text() + "\n\n"
            
    return text

In [ ]:
def get_npr_urls(soup):
    
    pattern = r'20[\d]{2}/[\d]{1,2}/[\d]{1,2}/[\d]+/'
    
    urls = []
    for link in soup.find_all('a', href=True):
        url = link['href']
        if (url.startswith('https://www.npr.org/')):
            if re.search(pattern, url):
                urls.append(url)
        
    return urls

In [ ]:
def get_title(soup):
    title = ""
    #find href with class == title, then get b
    for link in soup.find_all('a'):
        if link.has_attr('class'):
            if link['class'][0] == "title":
                title = link.findChildren('b')[0].get_text()
    
    return title

In [ ]:
def get_details(url):
    pattern = r'20[\d]{2}/[\d]{1,2}/[\d]{1,2}/[\d]+/'
    
    matchObject = re.search(pattern, url, flags=0)
    match_split = matchObject[0].split("/")
    date = match_split[0] + "-" + match_split[1] + "-" + match_split[2]
    article_id = match_split[3]
    
    return date, article_id

In [ ]:
##############
## 

start_url = "https://www.npr.org/"
traversed_urls = set()

to_traverse = set()
to_traverse.add(start_url)

        
while len(to_traverse) > 0 and article_count < max_traverse:

    ## Create a list of URLS to traverse
    
    ## Confirm that the url's have NOT been traversed before (query against DB)
    
    ## Pass that list to multi-threading
    
    ## Multi-threading should return dictionaries mapping to results and to discovered URLs
    
    ## Create class for DB that maintains unique set of to_traverse and traverse
    
    
    
    url = to_traverse.pop()
    traversed_urls.add(url)

    soup = BeautifulSoup(requests.get(url).text, 'html.parser')

    links = get_npr_urls(soup)

    urls = [x for x in get_npr_urls(soup) if x not in traversed_urls]
    to_traverse.update(urls)

    text = get_text(soup)

    if len(text) > 100:
        title = get_title(soup)
        date, article_id = get_details(url)

        if write:
            writer.writerow({"Link": url, "Title": title, "Date": date, "ID": article_id})
            with open (folder_out + article_id + ".txt", 'w') as article_hdl:
                article_hdl.write(text)


    article_count += 1